In [1]:
import os
import sys
import time
import json
from datetime import datetime

import numpy as np
import pandas as pd

#from models import models
from utils import read_df, read_numpy, write_df
import logging

from evaluate import rps

In [2]:
DATASET_PATH = './dataset/processed/train_v1'
TEST_DATA_FILENAME = 'test_data.csv'
GROUND_TRUTH_FILENAME = 'test_ground_truth.npy'
TRAIN_DATA_FILENAME = 'train_data.parquet'
TRAIN_DATA_PROCESSED_FILENAME = 'train_data_features.parquet'
ITEM_DATA_FILEPATH = './dataset/items_static_metadata_full.jl'

TRAIN_DATA_X_FILENAME = 'train_data_x.parquet'
TRAIN_DATA_Y_FILENAME = 'train_data_y.parquet'
TRAIN_DATA_X_PROCESSED_FILENAME = 'train_data_x_features.parquet'
TRAIN_DATA_Y_PROCESSED_FILENAME = 'train_data_y_features.parquet'

TEST_FROMTRAIN_DATA_LAST29_FILENAME = 'test_fromtrain_data_last29.parquet'
TEST_FROMTRAIN_DATA_LAST29_PROCESSED_FILENAME = 'test_fromtrain_data_last29_features.parquet'

TRAIN_DATA_X_PROCESSED_TSFRESH_FILENAME = 'train_data_x_features_tsfresh.parquet'
TEST_FROMTRAIN_DATA_LAST29_PROCESSED_TSFRESH_FILENAME = 'test_fromtrain_data_last29_features_tsfresh.parquet'

TRAIN_DATA_PROCESSEDV2_FILENAME = 'train_data_featuresv2.parquet'
TRAIN_DATA_X_PROCESSEDV2_FILENAME = 'train_data_x_featuresv2.parquet'
TEST_FROMTRAIN_DATA_LAST29V2_FILENAME = 'test_fromtrain_data_last29_featuresv2.parquet'

In [3]:
dateset_indexes = [0]
model_name = 'simple_first_30_days_fixed_spike'

In [4]:
dataset_index = dateset_indexes[0]
dataset_current_path = os.path.join(DATASET_PATH, str(dataset_index))

test_data_filepath = os.path.join(dataset_current_path, TEST_DATA_FILENAME)
ground_truth_filepath = os.path.join(dataset_current_path, GROUND_TRUTH_FILENAME)
train_data_filepath = os.path.join(dataset_current_path, TRAIN_DATA_FILENAME)
train_data_processed_filepath = os.path.join(dataset_current_path, TRAIN_DATA_PROCESSED_FILENAME)

train_data_x_filepath = os.path.join(dataset_current_path, TRAIN_DATA_X_FILENAME)
train_data_y_filepath = os.path.join(dataset_current_path, TRAIN_DATA_Y_FILENAME)
train_data_x_processed_filepath = os.path.join(dataset_current_path, TRAIN_DATA_X_PROCESSED_FILENAME)
train_data_y_processed_filepath = os.path.join(dataset_current_path, TRAIN_DATA_Y_PROCESSED_FILENAME)

test_fromtrain_data_last29_filepath = os.path.join(dataset_current_path, TEST_FROMTRAIN_DATA_LAST29_FILENAME)
test_fromtrain_data_last29_processed_filepath = os.path.join(dataset_current_path, TEST_FROMTRAIN_DATA_LAST29_PROCESSED_FILENAME)

train_data_x_processed_tsfresh_filepath = os.path.join(dataset_current_path, TRAIN_DATA_X_PROCESSED_TSFRESH_FILENAME)
test_fromtrain_data_last29_processed_tsfresh_filepath = os.path.join(dataset_current_path, TEST_FROMTRAIN_DATA_LAST29_PROCESSED_TSFRESH_FILENAME)

train_data_processedv2_filepath = os.path.join(dataset_current_path, TRAIN_DATA_PROCESSEDV2_FILENAME)
train_data_x_processedv2_filepath = os.path.join(dataset_current_path, TRAIN_DATA_X_PROCESSEDV2_FILENAME)
test_fromtrain_data_last29_processedv2_filepath = os.path.join(dataset_current_path, TEST_FROMTRAIN_DATA_LAST29V2_FILENAME)

In [5]:
ground_truth = read_numpy(ground_truth_filepath)

In [6]:
EVAL_PATH = './evaluations'

In [23]:
models = []
predictions = {}

for item in os.listdir(EVAL_PATH):
    if not item.endswith('.npy'):
        continue
    
    model_name = item[:-6]
    eval_filepath = os.path.join(EVAL_PATH, item)
    
    models.append(model_name)
    predictions[model_name] = np.load(eval_filepath)[0]

In [ ]:
from skopt import gp_minimize
def tune(params):
    #model_weights = {model_name:param for param, model_name in zip(params[:-2], models)}
    model_weights = {'xgboost_features2_v4_5_normalize': 7, 'xgboost_features2_v4_3_eliminate_features': 3, 'xgboost_features2_v4_4_select_3': 5, 'xgboost_features2_v4_6_more_target_feature': 1, 'xgboost_features2_v4_2_normalize_features': 5}
    print(model_weights, params[-2], params[-1])
    total = sum(params[:-2])
    pred = np.zeros(ground_truth.shape)
    for model_name in models:
        prediction = predictions[model_name]
        prediction = prediction + params[-2]*(prediction - params[-1]).clip(0, 1)
        pred += prediction*(model_weights[model_name]/total)
    
    pred = (pred/pred.sum(axis=1)[:,None]).round(4)
    
    return rps(pred, ground_truth)

#space = [(0, 10) for i in range(len(models))]
space.append((0.0, 1))
space.append((0.0, 1))
res = gp_minimize(tune, space, n_calls=1000, random_state=42, verbose=1, acq_optimizer="lbfgs", n_jobs=10)

Iteration No: 1 started. Evaluating function at random point.
{'xgboost_features2_v4_5_normalize': 7, 'xgboost_features2_v4_3_eliminate_features': 3, 'xgboost_features2_v4_4_select_3': 5, 'xgboost_features2_v4_6_more_target_feature': 1, 'xgboost_features2_v4_2_normalize_features': 5} 0.650888472948853 0.05641157902710027
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.7683
Function value obtained: 3.8369
Current minimum: 3.8369
Iteration No: 2 started. Evaluating function at random point.
{'xgboost_features2_v4_5_normalize': 7, 'xgboost_features2_v4_3_eliminate_features': 3, 'xgboost_features2_v4_4_select_3': 5, 'xgboost_features2_v4_6_more_target_feature': 1, 'xgboost_features2_v4_2_normalize_features': 5} 0.3998609717152556 0.046665663213615434
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.7963
Function value obtained: 3.8356
Current minimum: 3.8356
Iteration No: 3 started. Evaluating function at random point.
{'xgboost_features2_v4_5_nor

In [ ]:
#BEST 3.8379 {'xgboost_features2_v4_5_normalize': 7, 'xgboost_features2_v4_3_eliminate_features': 3, 'xgboost_features2_v4_4_select_3': 5, 'xgboost_features2_v4_6_more_target_feature': 1, 'xgboost_features2_v4_2_normalize_features': 5}

In [ ]:
# 3.8353 ? {'xgboost_features2_v4_5_normalize': 6, 'xgboost_features2_v4_3_eliminate_features': 5, 'xgboost_features2_v4_4_select_3': 10, 'xgboost_features2_v4_6_more_target_feature': 5, 'xgboost_features2_v4_2_normalize_features': 8} 0.2741145422657673 1/30